### Réapprentissage automatique

In [11]:
import sys
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider 
import csv
import pandas as pd
import numpy as np

In [12]:
# Cassandra metadata
dev_cassandra_host= 'dtl-cassandra01-d01'
dev_cassandra_port= 9200
dev_username= 'loaddata'
dev_password= 'DecujRiQuigByaibdednofVerr6Odij2'

# Cassandra Keyspaces
parcours_keyspace = 'parcours'
referentiel_keyspace = 'ref_data'

# Cassandra tables
insee_table = 'ref_insee'
prospect_table = 'referentiel_prospects'
valo_retro_one_month_table = 'retro_valo_client_un_mois'

# Elasticsearch hostname
dev_es_host= 'dtl-esmaster01-d01'

# Elasticsearch port
dev_es_port= 9200

# Path to pickle 
path_to_one_month_pickle = "/home/slazrak/projet-encours/dtl-encours/MODEL/ENCOURS_1MOIS/"

# One month model target name
one_month_target_name = 'ENCOURS_1MOIS'

In [13]:
class CassandraClient:

    def __init__(self, keyspace):
        self.auth_provider = None
        self.cluster = None
        self.session = None
        self.keyspace = keyspace
        self.create_session()
        self.prepare_statements()

    def create_session(self):
        self.auth_provider = PlainTextAuthProvider(username=dev_username, password=dev_password)
        self.cluster = Cluster(contact_points = [dev_cassandra_host], protocol_version=3, auth_provider=self.auth_provider)
        self.session = self.cluster.connect(parcours_keyspace)

    def get_session(self):
        return self.session

    def prepare_statements(self):
        ### prospects referential
        self.statement_get_prospects = self.session.prepare('select * from ' + prospect_table + ' WHERE csp>-1 ALLOW FILTERING')
        ### Get real "encours" from Retro 1 month  
        self.statement_get_valo_retro_one_month = self.session.prepare('select * from' + retro_valo_client_un__mois + 'WHERE encours>-1 ALLOW FILTERING')
        ### get Insee for enrichment
        self.statement_get_insee = self.session.prepare('select * from ' + referentiel_keyspace + '.'+ insee_table)
          

    def get_df_from_query(self,prepared_query):
        resultset = self.session.execute(prepared_query)
        resultset[0]
        df = pd.DataFrame()
        for num_col in range(len(resultset.column_names)):
            result_col = []
            for row in resultset:
                result_col.append(row[num_col])
            df[resultset.column_names[num_col]] = result_col
        return df



In [33]:
#cc = CassandraClient(parcours_keyspace)

### Get prospects' referential 
prospects_ref = cc.get_df_from_query(cc.statement_get_prospects)
#prospects_ref = pd.read_csv('/home/ubuntu/Bureau/ENCOURS_1MOIS_v2/ref_prospect.csv', sep=',')
prospects_ref.head()

,contactid,annee_mois,campagne,campaign,civilite,code_postal,code_postal_naissance,country_connexion_name,csp,date_naissance,...,mailing_accord,namesponsor,nature_cb,patrimoine,pays,pays_naissance,regime_matrimonial,revenus_annuels,service,timestamp
0,7280395968,201702,NaN,NaN,NaN,NaN,57200.0,NaN,4.0,NaN,...,NaN,NaN,NaN,0.0,NaN,FRA,4.0,1.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2017-02-12T20:08:39.301+0100
1,6952202606,201703,307.0,REC_Onlin_MC_Googl,MME,33450.0,33000.0,France,13.0,05/05/1976,...,0.0,NaN,NaN,1.0,FRA,FRA,2.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2017-03-09T15:01:12.021+0100
2,7359652653,201703,868.0,PAR_Onlin_Site _ECard,MLE,94130.0,NaN,France,-1.0,15/12/1986,...,0.0,BOUTTE,NaN,-1.0,FRA,NaN,-1.0,-1.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2017-03-01T08:48:44.504+0100
3,6884394982,201703,300.0,Intb_Banque_Autre,MR,44120.0,NaN,France,NaN,01/05/1982,...,0.0,NaN,NaN,NaN,FRA,NaN,NaN,NaN,monprofil.PROSPECT/CREER_FICHE_PROSPECT,2017-03-17T18:34:55.766+0100
4,5692919554,201702,456.0,REC_Liens textes autopromotion 456,MR,59150.0,59160.0,United Kingdom,2.0,06/12/1984,...,0.0,NaN,NaN,0.0,FRA,FRA,0.0,2.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2017-02-05T10:45:06.179+0100


In [34]:
### Get real 'encours' of customers from Retro 
retro = cc.get_df_from_query(cc.statement_get_valo_retro_one_month)
#retro = pd.read_csv('/home/ubuntu/Bureau/ENCOURS_1MOIS_v2/retro.csv', sep=',')
retro.head()

,id_dim_personne,annee_mois,annee_mois_m_x,cible_encours_predit,cible_reelle,date_entree_relation,delai_transfo,encours,id_dim_temps,identifiant_web,score_prospect_encours,x_mois_anterieurs_cumules
0,7367350880,201703,20170330,1,0,2017-02-28 00:00:00+0000,NaN,246.37,2017-02-28,46557377,0.750344,1
1,7423237535,201704,20170406,1,0,2017-03-09 00:00:00+0000,NaN,380.00,2017-03-09,89137569,0.853951,1
2,7576677073,201705,20170504,0,0,2017-04-05 00:00:00+0000,NaN,220.96,2017-04-05,19787090,0.219903,1
3,7319308930,201703,20170316,1,0,2017-02-19 00:00:00+0000,NaN,10.00,2017-02-19,54097163,0.577456,1
4,7478891061,201704,20170419,1,0,2017-03-20 00:00:00+0000,NaN,0.00,2017-03-20,76707761,0.765792,1


In [35]:
#Delete Unnecessary fields from retro
del retro['annee_mois']
del retro['annee_mois_m_x']
del retro['cible_encours_predit']
del retro['date_entree_relation']
del retro['delai_transfo']
del retro['id_dim_temps']
del retro['identifiant_web']
del retro['score_prospect_encours']
del retro['x_mois_anterieurs_cumules']
retro['cible'] = retro['cible_reelle']
del retro['cible_reelle']

In [36]:
retro.head()

,id_dim_personne,encours,cible
0,7367350880,246.37,0
1,7423237535,380.00,0
2,7576677073,220.96,0
3,7319308930,10.00,0
4,7478891061,0.00,0


In [39]:
### Merge tables
merge_1 = pd.merge(prospects_ref,retro, right_on='id_dim_personne', left_on='contactid')
merge_1.head()

,contactid,annee_mois,campagne,campaign,civilite,code_postal,code_postal_naissance,country_connexion_name,csp,date_naissance,...,patrimoine,pays,pays_naissance,regime_matrimonial,revenus_annuels,service,timestamp,id_dim_personne,encours,cible
0,7304061587,201702,868.0,PAR_Onlin_Site _ECard,MLE,91100.0,97114.0,France,12.0,04/08/1992,...,0.0,FRA,FRA,0.0,0.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2017-02-16T09:22:09.464+0100,7304061587,378.5,0
1,7416732641,201703,868.0,PAR_Onlin_Site _ECard,MLE,38210.0,38000.0,France,4.0,09/01/1984,...,0.0,FRA,FRA,0.0,1.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2017-03-08T22:51:49.807+0100,7416732641,380.0,0
2,7280327415,201702,300.0,Intb_Banque_Autre,MR,57920.0,57100.0,France,16.0,09/03/1991,...,0.0,FRA,FRA,0.0,1.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2017-02-12T18:32:19.909+0100,7280327415,300.0,0
3,7591253811,201704,868.0,PAR_Onlin_Site _ECard,MME,94000.0,6000.0,France,2.0,20/06/1978,...,1.0,FRA,FRA,2.0,1.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2017-04-07T14:43:45.741+0200,7591253811,300.0,0
4,7485895222,201703,868.0,PAR_Onlin_Site _ECard,MR,75020.0,6000.0,Spain,3.0,27/06/1970,...,2.0,FRA,FRA,0.0,3.0,monprofil.CONTACT_MANAGEMENT/amendContactInfor...,2017-03-21T13:11:59.027+0100,7485895222,1580.0,1


In [42]:
### Get INSEE for enrichment 
#cc2 = CassandraClient(referentiel_keyspace)
insee_ref = cc.get_df_from_query(cc2.statement_get_insee)
#insee_ref = pd.read_csv('/home/ubuntu/Bureau/ENCOURS_1MOIS_v2/Referentiel1.csv', sep = ',')
#insee_ref['code_postal'] = insee_ref['Code_geographique']
#del insee_ref['Libelle_geographique'] 
#del insee_ref['Code_geographique'] 
insee_ref.head()

,Salaire_net_horaire_moyen_en_2014_euro,code_postal
0,"13,7",1004
1,"13,5",1007
2,"13,5",1014
3,"12,9",1024
4,"13,0",1025


In [43]:
### Merge previous table with insee
merge_2 = pd.merge(merge_1,insee_ref, right_on= 'code_postal', left_on= 'code_postal')
merge_2.head()

,contactid,annee_mois,campagne,campaign,civilite,code_postal,code_postal_naissance,country_connexion_name,csp,date_naissance,...,pays,pays_naissance,regime_matrimonial,revenus_annuels,service,timestamp,id_dim_personne,encours,cible,Salaire_net_horaire_moyen_en_2014_euro


In [44]:
merge_2 = merge_2.set_index('contactid')

In [46]:
del merge_2['cible']

In [47]:
#Fix encours of good costumers (1200) 
merge_2['cible_seuil_1200'] = merge_2['encours'].map(lambda x: 0 if x < 1200.00 else 1).astype(int)
merge_2[['encours','cible_seuil_1200']].head()

,encours,cible_seuil_1200
contactid,,


In [48]:
### Data management
def data_management(df):
    
    ### Get flag parrain from namesponsor
    df['Parrain'] = df['namesponsor'].map(lambda x: 0 if pd.isnull(x) else 1).astype(int)
    del df['firstnamesponsor'] 
    del df['namesponsor'] 
    
    ### Get domain from mail
    df['mail'] = df.mail.replace ([np.nan], '')
    df['domain'] = df['mail'].map(lambda x: x.split("@")[1] if x else None)
    df['domain'] = df.domain.str.lower()   

    ### Get Sex from Civility
    df['Sexe'] = df['civilite'].map(get_sex)
    
    ### Delete useless cols
    del df['civilite']
    del df['mail']
    del df['annee_mois']
    del df['mailing_accord']
    del df['id_dim_temps']
    del df['id_dim_personne']
    del df['encours']
    del df['timestamp']
    del df['service']
    del df['campaign']
    #del df['contactid']
    del df['pays']
    del df['debit_cb']
    del df['code_postal']
    del df['nature_cb']
    
    ### Impute empty rows  
    df['date_naissance'] = df['date_naissance'].replace('','01/01/1800')
    df = df.replace ([np.nan,None,''], -2)
    
    ### Get age from birthdate
    get_age(df)

    ### transform to numeric when possible
    df = df.apply(lambda x: pd.to_numeric(x, errors='ignore'))
    
    ### filter the most reccurent ones
    df.loc[df['country_connexion_name'].value_counts()[df['country_connexion_name']].values < 40, 'country_connexion_name'] = 'other'
    df.loc[df['domain'].value_counts()[df['domain']].values < 40, 'domain'] = 'other'
    df.loc[df['pays_naissance'].value_counts()[df['pays_naissance']].values < 40, 'pays_naissance'] = 'other'
     
    ### Get country connexion name
    df = process_country_connexion (df)
    
    ### Get pays de naissance
    df = process_pays_naissance(df)
    #df['pays_de_naissance'] = df['pays_naissance'].map(process_pays_naissance)
    #del df['pays_naissance']
    
    ### Process the domain
    df = process_domain (df)
    
    ### Drop NaN 
    #df = df.dropna()
    df = df.replace ([np.nan,None,''], -2)
    
    features_df = df.drop('cible_seuil_1200', axis=1)
    target = df['cible_seuil_1200']
    return features_df, target, df


def get_sex(x):
    if x == "0":
        return 1
    elif (x == "1"):
        return 0
    elif (x == "2"):
        return 0
    elif (x == "MR"):
        return 1
    elif (x == "MLE"):
        return 0
    elif (x == "MME"):
        return 0
    else :
        return -1

def get_age(X):
    from datetime import datetime
    now = datetime.now()
    X['AGE'] = X['date_naissance'].dropna()
    X['AGE'] = pd.to_datetime(X['AGE'],errors='coerce')
    X['AGE'] = X['AGE'].map(lambda x : now.year-x.year  if now.month-x.month>0 else now.year - x.year -1 )
    del X['date_naissance']

def process_country_connexion(df):
    """
    Process country_connexion
    """
    dict_ = {'France':1, 'United Kingdom':2, 'other': 3, 'Germany': 4, 'Switzerland': 5, 'Netherlands': 6, 'Europe': 7, 'United States': 8, 'Reunion': 9, -2: -2}
    df['country_connexion'] = df['country_connexion_name'].map(dict_)
    del df['country_connexion_name']
    return df
    
def process_pays_naissance(df):
    """
    Process pays naissance
    """
    dict_ = {'FRA':1, 'ITA':2, 'other': 3, 'DZA': 4, 'BEN': 5, 'ESP': 6, 'VNM': 7, 'DEU': 8, 'MAR': 9, 'CIV': 10, 'CMR': 11, 'GTO': 12,
            'SEN': 13,'BEL': 14, 'CHN': 15, 'ROU':16, 'BRA': 17, 'MDG': 18,
            'PRT': 19, 'GBR': 20, 'LBN': 21, 'TUR': 22, 'IND': 23}
    df['pays_de_naissance'] = df['pays_naissance'].map(dict_)
    del df['pays_naissance']
    return df
  
def process_domain(df):
    """
    Process the domain features
    """
    dict_ = {'yahoo.fr':0, 'hotmail.fr':1, 'hotmail.com': 2, 'gmail.com': 3, 'orange.fr': 4, 'outlook.com': 5, 'free.fr': 6, 'laposte.net': 7, 'other': 8, 'neuf.fr': 9, 'wanadoo.fr': 10, 'me.com': 11, 'ymail.com': 12, 'sfr.fr': 13, 'live.fr': 14, 'bbox.fr': 15, 'outlook.fr': 16, 'msn.com': 17, 'yahoo.com': 18, 'aol.com': 19, 'icloud.com': 20, 'cegetel.net': 21, 'club-internet.fr': 22}
    df['domain'] = df['domain'].map(dict_)
    return df

In [50]:
X, y, data= data_management(merge_2)

In [56]:
print ('Types', data.dtypes)
print ('X.shape', X.shape)
print ('y.shape', y.shape)

('Types', campagne                                  float64
code_postal_naissance                     float64
csp                                       float64
flag_banque_principale                    float64
flag_epargne                              float64
patrimoine                                float64
regime_matrimonial                        float64
revenus_annuels                           float64
Salaire_net_horaire_moyen_en_2014_euro     object
cible_seuil_1200                            int64
Parrain                                     int64
domain                                      int64
Sexe                                       object
AGE                                        object
country_connexion                           int64
pays_de_naissance                           int64
dtype: object)
('X.shape', (0, 15))
('y.shape', (0,))


In [58]:
# Modèle : nearest_centroid
from sklearn.neighbors.nearest_centroid import NearestCentroid
classfier = NearestCentroid(metric='euclidean', shrink_threshold = 1.5)
classifier.fit(X,y)

In [60]:
## save the model to disk
import cPickle
with open('Model_v2.pkl', 'wb') as f:
    cPickle.dump(classifier, f)